<a href="https://colab.research.google.com/github/nickelodeon20/Test-Model-Performances/blob/main/ShallowFBCSPNet_BCI_IV2a_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Authors: Lukas Gemein <l.gemein@gmail.com>
#          Hubert Banville <hubert.jbanville@gmail.com>
#          Simon Brandt <simonbrandt@protonmail.com>
#          Daniel Wilson <dan.c.wil@gmail.com>
#
# License: BSD (3-clause)
!pip install braindecode # Install braindecode package
from braindecode.datasets import MOABBDataset

In [10]:
subject_id = 3

!pip install moabb
dataset = MOABBDataset(dataset_name="BNCI2014_001", subject_ids=[subject_id])


In [11]:
from numpy import multiply

from braindecode.preprocessing import (Preprocessor,
                                       exponential_moving_standardize,
                                       preprocess)

low_cut_hz = 4.  # low cut frequency for filtering
high_cut_hz = 38.  # high cut frequency for filtering
# Parameters for exponential moving standardization
factor_new = 1e-3
init_block_size = 1000
# Factor to convert from V to uV
factor = 1e6

# Define a named function for volt-to-microvolt conversion
def convert_to_microvolts(data, factor=factor):
    return multiply(data, factor)

preprocessors = [
    Preprocessor('pick_types', eeg=True, meg=False, stim=False),  # Keep EEG sensors
    Preprocessor(convert_to_microvolts,  apply_on_array=True),  # Convert from V to uV using the named function    Preprocessor('filter', l_freq=low_cut_hz, h_freq=high_cut_hz),  # Bandpass filter
    Preprocessor(exponential_moving_standardize,  # Exponential moving standardization
                 factor_new=factor_new, init_block_size=init_block_size)
]

# Transform the data
preprocess(dataset, preprocessors, n_jobs=-1)

In [12]:
from braindecode.preprocessing import create_windows_from_events

trial_start_offset_seconds = -0.5
# Extract sampling frequency, check that they are same in all datasets
sfreq = dataset.datasets[0].raw.info['sfreq']
assert all([ds.raw.info['sfreq'] == sfreq for ds in dataset.datasets])
# Calculate the trial start offset in samples.
trial_start_offset_samples = int(trial_start_offset_seconds * sfreq)

# Create windows using braindecode function for this. It needs parameters to define how
# trials should be used.
windows_dataset = create_windows_from_events(
    dataset,
    trial_start_offset_samples=trial_start_offset_samples,
    trial_stop_offset_samples=0,
    preload=True,
)

Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']


In [40]:
from braindecode.datasets import BaseConcatDataset

# Split the dataset into training and validation set.

splitted = windows_dataset.split('session')
train_set = splitted['0train']  # Session train
#print(train_set[0][0])
print("------------------------------")
valid_set = splitted['1test']  # Session evaluation
#print(valid_set[0][0])
print("------------------------------")


# Get the list of datasets within valid_set
valid_set_datasets = valid_set.datasets

# Create new split indices within the valid_set
n_valid_trials = len(valid_set_datasets)
valid_subset1_ids = list(range(0, int(0.8 * n_valid_trials)))  # First half of valid_set
valid_subset2_ids = list(range(int(0.2 * n_valid_trials), n_valid_trials))  # Second half of valid_set

# Create subsets of valid_set based on indices
valid_subset1 = BaseConcatDataset([valid_set_datasets[i] for i in valid_subset1_ids])
valid_subset2 = BaseConcatDataset([valid_set_datasets[i] for i in valid_subset2_ids])



# Update the valid_set with the new subsets
valid_set.datasets = [valid_subset1, valid_subset2]

valid_set_l, valid_set_w, valid_set_h = valid_set.datasets[0], valid_set.datasets[0][0], valid_set.datasets[0][0][0]
print("L x W x H = " + str(len(valid_set_l)) + " x " + str(len(valid_set_w)) + " x " + str(len(valid_set_h)))
print("------------------------------")
valid_set_l, valid_set_w, valid_set_h = valid_set.datasets[0], valid_set.datasets[0][0], valid_set.datasets[0][0][0]
print("L x W x H = " + str(len(valid_set_l)) + " x " + str(len(valid_set_w)) + " x " + str(len(valid_set_h)))
print("------------------------------")
valid_set_l, valid_set_w, valid_set_h = valid_set.datasets[0], valid_set.datasets[0][0], valid_set.datasets[0][0][0]
print("L x W x H = " + str(len(valid_set_l)) + " x " + str(len(valid_set_w)) + " x " + str(len(valid_set_h)))
"""
test_size = (len(valid_set.datasets)) // 3

# Create the test set using slicing on the underlying datasets
test_set = valid_set.datasets[:test_size]
test_set = type(valid_set)(test_set) # To maintain BaseConcatDataset type
print(test_set[0][0])
print("------------------------------")

# Update the validation set by excluding the test set portion
valid_set = valid_set.datasets[test_size:]
valid_set = type(valid_set)(valid_set)  # To maintain BaseConcatDataset type
print(valid_set[0][0])
print("------------------------------")
"""


------------------------------
------------------------------
[[ 0.22230309 -0.04833731 -1.2568867  ...  0.85724396  2.1186037
   0.48831838]
 [ 0.17376888  0.14103435 -1.4208701  ...  0.7195906   2.7172763
   1.7720041 ]
 [ 0.18767089 -0.09634947 -1.2768091  ...  0.8923618   2.3300319
   0.99394417]
 ...
 [ 0.70805854  0.5330354  -0.79146415 ...  0.59256047  2.0856087
   0.6110683 ]
 [ 0.61362165  0.3323485  -0.9576285  ...  0.6219022   2.2601748
   0.8375986 ]
 [ 0.657738    0.45626017 -0.7251323  ...  0.47373986  1.8942503
   0.72885215]]
------------------------------
[[-0.76391214 -0.85032845 -1.2509857  ... -0.48516664 -0.13433345
  -0.19352888]
 [-1.07877    -1.173073   -1.6210124  ... -0.8607335  -0.26098868
  -0.74236536]
 [-0.8934356  -1.057702   -1.4941813  ... -0.9723431  -0.5593034
  -0.79070634]
 ...
 [-0.57635486 -0.4676753  -1.4729612  ... -0.30570632  0.32585996
  -0.10835916]
 [-0.4150864  -0.3560559  -1.1395516  ... -0.30622593  0.43472254
   0.01227009]
 [-0.5184523

'\ntest_size = (len(valid_set.datasets)) // 3\n\n# Create the test set using slicing on the underlying datasets\ntest_set = valid_set.datasets[:test_size]\ntest_set = type(valid_set)(test_set) # To maintain BaseConcatDataset type\nprint(test_set[0][0])\nprint("------------------------------")\n\n# Update the validation set by excluding the test set portion\nvalid_set = valid_set.datasets[test_size:]\nvalid_set = type(valid_set)(valid_set)  # To maintain BaseConcatDataset type\nprint(valid_set[0][0])\nprint("------------------------------")\n'

In [ ]:
import torch

from braindecode.models import ShallowFBCSPNet
from braindecode.util import set_random_seeds

cuda = torch.cuda.is_available()  # check if GPU is available, if True chooses to use it
device = 'cuda' if cuda else 'cpu'
if cuda:
    torch.backends.cudnn.benchmark = True
# Set random seed to be able to roughly reproduce results
# Note that with cudnn benchmark set to True, GPU indeterminism
# may still make results substantially different between runs.
# To obtain more consistent results at the cost of increased computation time,
# you can set `cudnn_benchmark=False` in `set_random_seeds`
# or remove `torch.backends.cudnn.benchmark = True`
seed = 20200220
set_random_seeds(seed=seed, cuda=cuda)

n_classes = 4
classes = list(range(n_classes))
# Extract number of chans and time steps from dataset
n_chans = train_set[0][0].shape[0]
input_window_samples = train_set[0][0].shape[1]

model = ShallowFBCSPNet(
    n_chans,
    n_classes,
    input_window_samples=input_window_samples,
    final_conv_length='auto',
)

# Display torchinfo table describing the model
print(model)

# Send model to GPU
if cuda:
    model = model.cuda()

Layer (type (var_name):depth-idx)        Input Shape               Output Shape              Param #                   Kernel Shape
ShallowFBCSPNet (ShallowFBCSPNet)        [1, 22, 1125]             [1, 4]                    --                        --
├─Ensure4d (ensuredims): 1-1             [1, 22, 1125]             [1, 22, 1125, 1]          --                        --
├─Rearrange (dimshuffle): 1-2            [1, 22, 1125, 1]          [1, 1, 1125, 22]          --                        --
├─CombinedConv (conv_time_spat): 1-3     [1, 1, 1125, 22]          [1, 40, 1101, 1]          36,240                    --
├─BatchNorm2d (bnorm): 1-4               [1, 40, 1101, 1]          [1, 40, 1101, 1]          80                        --
├─Expression (conv_nonlin_exp): 1-5      [1, 40, 1101, 1]          [1, 40, 1101, 1]          --                        --
├─AvgPool2d (pool): 1-6                  [1, 40, 1101, 1]          [1, 40, 69, 1]            --                        [75, 1]
├─Express

/usr/local/lib/python3.11/dist-packages/braindecode/models/base.py:23: UserWarning: ShallowFBCSPNet: 'input_window_samples' is depreciated. Use 'n_times' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/braindecode/models/base.py:180: UserWarning: LogSoftmax final layer will be removed! Please adjust your loss function accordingly (e.g. CrossEntropyLoss)!
  warnings.warn("LogSoftmax final layer will be removed! " +


In [ ]:
from skorch.callbacks import LRScheduler
from skorch.helper import predefined_split

from braindecode import EEGClassifier

# We found these values to be good for the shallow network:
lr = 0.0625 * 0.01
weight_decay = 0

# For deep4 they should be:
# lr = 1 * 0.01
# weight_decay = 0.5 * 0.001

batch_size = 64
n_epochs = 50

clf = EEGClassifier(
    model,
    criterion=torch.nn.NLLLoss,
    optimizer=torch.optim.AdamW,
    train_split=predefined_split(valid_set),  # using valid_set for validation
    optimizer__lr=lr,
    optimizer__weight_decay=weight_decay,
    batch_size=batch_size,
    callbacks=[
        "accuracy", ("lr_scheduler", LRScheduler('CosineAnnealingLR', T_max=n_epochs - 1)),
    ],
    device=device,
    classes=classes,
)
# Model training for the specified number of epochs. `y` is None as it is
# already supplied in the dataset.
_ = clf.fit(train_set, y=None, epochs=n_epochs)

ModuleNotFoundError: No module named 'skorch'

Line Graph

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.lines import Line2D

# Extract loss and accuracy values for plotting from history object
results_columns = ['train_loss', 'valid_loss', 'train_accuracy', 'valid_accuracy']
df = pd.DataFrame(clf.history[:, results_columns], columns=results_columns,
                  index=clf.history[:, 'epoch'])

# get percent of misclass for better visual comparison to loss
df = df.assign(train_misclass=100 - 100 * df.train_accuracy,
               valid_misclass=100 - 100 * df.valid_accuracy)

fig, ax1 = plt.subplots(figsize=(8, 3))
df.loc[:, ['train_loss', 'valid_loss']].plot(
    ax=ax1, style=['-', ':'], marker='o', color='tab:blue', legend=False, fontsize=14)

ax1.tick_params(axis='y', labelcolor='tab:blue', labelsize=14)
ax1.set_ylabel("Loss", color='tab:blue', fontsize=14)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

df.loc[:, ['train_misclass', 'valid_misclass']].plot(
    ax=ax2, style=['-', ':'], marker='o', color='tab:red', legend=False)
ax2.tick_params(axis='y', labelcolor='tab:red', labelsize=14)
ax2.set_ylabel("Misclassification Rate [%]", color='tab:red', fontsize=14)
ax2.set_ylim(ax2.get_ylim()[0], 85)  # make some room for legend
ax1.set_xlabel("Epoch", fontsize=14)

# where some data has already been plotted to ax
handles = []
handles.append(Line2D([0], [0], color='black', linewidth=1, linestyle='-', label='Train'))
handles.append(Line2D([0], [0], color='black', linewidth=1, linestyle=':', label='Valid'))
plt.legend(handles, [h.get_label() for h in handles], fontsize=14)
plt.tight_layout()

Training Confusion Matrices


In [ ]:
from sklearn.metrics import confusion_matrix

from braindecode.visualization import plot_confusion_matrix

# generate confusion matrices
# get the targets
y_true = valid_set.get_metadata().target
y_pred = clf.predict(valid_set)

# generating confusion matrix
confusion_mat = confusion_matrix(y_true, y_pred)

# add class labels
# label_dict is class_name : str -> i_class : int
label_dict = windows_dataset.datasets[0].window_kwargs[0][1]['mapping']
# sort the labels by values (values are integer class labels)
labels = [k for k, v in sorted(label_dict.items(), key=lambda kv: kv[1])]

# plot the basic conf. matrix
plot_confusion_matrix(confusion_mat, class_names=labels)


ModuleNotFoundError: No module named 'braindecode'

Test Confusion Matrices

In [ ]:
# generate confusion matrices
# get the targets
y_true_test = test_set.get_metadata().target
y_pred_test = clf.predict(test_set)

# generating confusion matrix
confusion_mat_test = confusion_matrix(y_true_test, y_pred_test)

# add class labels
# label_dict is class_name : str -> i_class : int
label_dict = windows_dataset.datasets[0].window_kwargs[0][1]['mapping']
# sort the labels by values (values are integer class labels)
labels = [k for k, v in sorted(label_dict.items(), key=lambda kv: kv[1])]

# plot the basic conf. matrix
plot_confusion_matrix(confusion_mat_test, class_names=labels